In [1]:
import copy

import isambard

# Helix Pair

In [3]:
test_structure = isambard.ampal.convert_pdb_to_ampal('5ced.pdb')

In [6]:
sol1 = test_structure[1]

In [7]:
sol1.helices.sequences

['KALNEAAEQ',
 'LAKVKNLVQ',
 'PLMNAAM',
 'LDIVKFLLS',
 'ALAFAVT',
 'YDVAEELIK',
 'LFMRAA',
 'KKTAESILAK',
 'ALFAVARY',
 'PADIDFLIK',
 'ALDVAKE',
 'QDTAKALSK']

In [16]:
rep_hels = sol1.helices[4:6]

In [17]:
rep_hels.relabel_all()

In [18]:
with open('/Users/ChrisWood/Desktop/blah.pdb', 'w') as outf:
    outf.write(rep_hels.pdb)

In [19]:
hel_pair = isambard.topology.HelixPair(aas=(7, 9))

In [20]:
hel_pair.sequences

['GGGGGGG', 'GGGGGGGGG']

In [21]:
rep_hels.sequences

['ALAFAVT', 'YDVAEELIK']

In [14]:
hel_pair.relabel_all()

In [22]:
hel_pair.pack_new_sequences(['LAFAVTN', 'YDVAEELIK'])

In [27]:
hel_pair.sequences

['LAFAVTN', 'YDVAEELIK']

In [24]:
class HelPairOpt(isambard.topology.HelixPair):
    def __init__(self, rad, zshift, phi1, phi2, sp, op):
        super().__init__(aas=(7, 9), axis_distances=(-rad, rad), z_shifts=(0, zshift),
                         phis=(phi1, phi2), splays=(0, sp), off_plane=(0, op))
        self.relabel_all()

In [25]:
isambard.external_programs.run_profit(rep_hels.pdb, hel_pair.pdb, path1=False, path2=False)

[4.621, 4.542, 5.796]

In [28]:
opt = isambard.optimisation.OptDE_rmsd(
    HelPairOpt, rep_hels.pdb)
opt.parameters(hel_pair.sequences,
          [3, 0, 0, 0, 0, 180],
          [3, 6, 180, 180, 45, 90],
          ['var0', 'var1', 'var2', 'var3', 'var4', 'var5'])
opt.run_de(50, 50, 3)

Starting minimisation (2016-07-06 11:53:38)
gen	evals	avg    	std     	min  	max  
0  	50   	3.92018	0.791047	1.718	4.959
1  	50   	3.58168	0.75757 	1.718	4.713
2  	50   	3.24054	0.743921	1.173	4.457
3  	50   	2.98328	0.681866	1.173	4.107
4  	50   	2.68406	0.666347	0.84 	3.821
5  	50   	2.46702	0.69223 	0.84 	3.481
6  	50   	2.04302	0.603826	0.84 	3.25 
7  	50   	1.71052	0.517575	0.744	3.172
8  	50   	1.42712	0.405186	0.675	2.381
9  	50   	1.21512	0.273443	0.577	1.742
10 	50   	1.07134	0.268004	0.545	1.742
11 	50   	0.97452	0.215966	0.545	1.531
12 	50   	0.89056	0.179743	0.545	1.224
13 	50   	0.8275 	0.153742	0.545	1.128
14 	50   	0.78234	0.140428	0.545	1.053
15 	50   	0.71992	0.135715	0.527	1.053
16 	50   	0.67194	0.102871	0.527	0.978
17 	50   	0.62508	0.0607552	0.503	0.759
18 	50   	0.58616	0.0533402	0.478	0.705
19 	50   	0.55864	0.0476725	0.478	0.685
20 	50   	0.53778	0.0404669	0.439	0.611
21 	50   	0.5139 	0.0361454	0.434	0.611
22 	50   	0.49666	0.0353936	0.428	0.607
23 	50   	0.47

In [29]:
best_params = opt.parse_individual(opt.halloffame[0])

In [30]:
best = HelPairOpt(*best_params)
best.pack_new_sequences(hel_pair.sequences)

In [31]:
with open('/Users/ChrisWood/Desktop/blah2.pdb', 'w') as outf:
    outf.write(best.pdb)

# Solenoid

In [42]:
target = isambard.ampal.Assembly()

In [43]:
test_structure = isambard.ampal.convert_pdb_to_ampal('5ced.pdb')

In [44]:
cha = test_structure[1]

In [45]:
cha.sequence

'GKSSKALNEAAEQGDLAKVKNLVQKNKIDLNAQDETGMTPLMNAAMGGNLDIVKFLLSKKVNLELKNNGGETALAFAVTNDAYDVAEELIKAGANVDIIVAGDEGDTLFMRAAQNNKKTAESILAKNKSLINKANTLGETALFAVARYGTPADIDFLIKKGADLKLKNKKGQTALDVAKEASNQDTAKALSKKK'

In [46]:
target.append(cha.get_slice_from_res_id('33', '39'))

In [47]:
target.append(cha.get_slice_from_res_id('42', '50'))

In [48]:
target.append(cha.get_slice_from_res_id('67', '73'))

In [49]:
target.append(cha.get_slice_from_res_id('76', '84'))

In [50]:
target.append(cha.get_slice_from_res_id('100', '106'))

In [51]:
target.append(cha.get_slice_from_res_id('109', '117'))

In [52]:
target.append(cha.get_slice_from_res_id('134', '140'))

In [53]:
target.append(cha.get_slice_from_res_id('143', '151'))

In [54]:
target.sequences

['LNEAAEQ',
 'LAKVKNLVQ',
 'LMNAAMG',
 'LDIVKFLLS',
 'LAFAVTN',
 'YDVAEELIK',
 'LFMRAAQ',
 'KKTAESILA']

In [55]:
rep_unit = HelPairOpt(*best_params)

In [56]:
class SolenoidOpt(isambard.topology.Solenoid):
    def __init__(self, in_ru, repeats, rad, rise, rot_ang, xr, yr, zr):
        ru = copy.deepcopy(in_ru)
        ru.rotate(xr, (1, 0, 0))
        ru.rotate(yr, (0, 1, 0))
        ru.rotate(zr, (0, 0, 1))
        super().__init__(ru, repeats, rad, rise, rot_ang, 'left')

In [57]:
target.relabel_polymers()

In [60]:
opt2 = isambard.optimisation.OptDE_rmsd(
    SolenoidOpt, target.pdb)
opt2.parameters(target.sequences,
          [35, 10, 0, 0, 0, 0],
          [20, 10.0, 200, 200, 200, 200],
          [rep_unit, 4, 'var0', 'var1', 'var2', 'var3', 'var4', 'var5'])
opt2.run_de(50, 50, 3)

Starting minimisation (2016-07-06 12:46:44)
gen	evals	avg    	std   	min  	max  
0  	50   	17.4172	10.554	4.287	47.28
1  	50   	12.816 	7.07653	4.271	32.493
2  	50   	9.67118	5.07511	4.04 	22.361
3  	50   	8.31146	4.0416 	4.04 	20.962
4  	50   	6.5904 	2.58057	3.983	16.019
5  	50   	5.45348	1.67531	2.881	11.478
6  	50   	4.88162	1.26224	2.222	9.119 
7  	50   	4.59456	0.994539	2.222	7.012 
8  	50   	4.26718	0.846437	2.222	5.942 
9  	50   	3.84306	0.805185	2.222	5.498 
10 	50   	3.71476	0.806554	2.222	5.303 
11 	50   	3.57308	0.768325	2.222	5.303 
12 	50   	3.39364	0.669133	2.222	4.904 
13 	50   	3.10748	0.508661	2.222	4.255 
14 	50   	2.9939 	0.440154	2.192	4.105 
15 	50   	2.83298	0.413731	2.192	3.833 
16 	50   	2.72926	0.370741	2.192	3.833 
17 	50   	2.6216 	0.335714	2.087	3.833 
18 	50   	2.51146	0.270058	2.087	3.195 
19 	50   	2.4657 	0.251285	2.087	3.195 
20 	50   	2.40436	0.220913	2.087	3.195 
21 	50   	2.34552	0.157238	2.087	2.716 
22 	50   	2.30144	0.140528	2.087	2.716 
23 	50  

In [61]:
best = SolenoidOpt(*opt2.parse_individual(opt2.halloffame[0]))

In [62]:
best.pack_new_sequences(target.sequences)

In [63]:
[x.id for x in best]

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [64]:
[x.id for x in target]

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [65]:
with open('/Users/ChrisWood/Desktop/blah3.pdb', 'w') as outf:
    outf.write(best.pdb)

```
ignoremissing
align
atoms ca
fit
atoms n,ca,c,o
fit
atoms *
fit
quit```